# Week 6 Major Group Assignment: Advanced SQL-Python E-commerce Analytics

## 🎯 Assignment Overview

**Business Context**: You are a data analytics team at Olist, Brazil's largest e-commerce platform. The executive team needs comprehensive insights to drive strategic decisions for 2024. Your team must combine advanced data analysis techniques with interactive visualizations to deliver actionable business intelligence.

**Group Size**: 3-4 students per group
**Duration**: 2 weeks
**Deliverables**: 
- This completed Jupyter notebook with analysis
- 10-minute group presentation
- Executive summary report (2-3 pages)

## 📚 Learning Objectives

This assignment integrates **Week 6 topics**:
- **Wednesday**: Advanced pandas operations, data manipulation, and feature engineering
- **Thursday**: SQL-Python integration, database connections, and hybrid analytics
- **Interactive Visualizations**: Plotly dashboards and storytelling with data

## 🏢 Business Scenario

Olist's executive team is preparing for their 2024 strategic planning meeting. They need data-driven insights to answer critical business questions:

1. **Customer Strategy**: Which customer segments drive the most value?
2. **Marketing Optimization**: Which acquisition channels provide the best ROI?
3. **Product Portfolio**: What product categories should we prioritize?
4. **Geographic Expansion**: Which regions offer the biggest growth opportunities?
5. **Operational Excellence**: How can we improve delivery performance and customer satisfaction?

Your team's analysis will directly influence million-dollar strategic decisions.

## 🗄️ Data Sources

You have access to Olist's complete dataset through **Supabase PostgreSQL**:
- **E-commerce Data**: 100k+ orders, customers, products, reviews
- **Marketing Funnel**: Lead generation, conversions, channel performance
- **Geographic Data**: Brazilian state and city information

**Database Connection**: Use the provided Supabase credentials from Thursday's lesson.

## 📋 Assignment Structure

### Part 1: Data Foundation (20 points)
- Establish PostgreSQL connection using SQLAlchemy
- Explore database schema and data quality
- Create comprehensive data quality report

### Part 2: Advanced Analytics (35 points)
- **Customer Segmentation**: Advanced pandas analysis with RFM modeling
- **Marketing Channel Analysis**: SQL-Python integration for ROI calculation
- **Geographic Intelligence**: Regional performance and expansion opportunities
- **Product Performance**: Category analysis with recommendation engine logic

### Part 3: Interactive Business Intelligence (25 points)
- Create interactive Plotly dashboard with 5+ visualizations
- Implement filters and cross-chart interactions
- Design executive-level presentation visuals

### Part 4: Strategic Recommendations (20 points)
- Generate actionable business insights
- Quantify impact and ROI of recommendations
- Create implementation roadmap

---

## 🚀 Getting Started

**Step 1**: Form your team and assign roles:
- **Data Engineer**: Database connections and data pipeline
- **Analyst**: Advanced pandas analysis and statistical modeling
- **Visualization Specialist**: Interactive dashboards and storytelling
- **Business Strategist**: Insights translation and recommendations

**Step 2**: Set up your environment and establish database connection

In [ ]:
# Setup and imports
import pandas as pd
import numpy as np
import sqlalchemy
from sqlalchemy import create_engine, text, inspect
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import warnings
import os
from dotenv import load_dotenv

warnings.filterwarnings('ignore')

# Team Information
TEAM_INFO = {
    'team_name': 'YOUR_TEAM_NAME',
    'members': [
        'Student 1 Name - Role',
        'Student 2 Name - Role', 
        'Student 3 Name - Role',
        'Student 4 Name - Role (if applicable)'
    ],
    'submission_date': 'YYYY-MM-DD'
}

print(f"🎯 Team: {TEAM_INFO['team_name']}")
print(f"👥 Members: {len(TEAM_INFO['members'])}")
for member in TEAM_INFO['members']:
    print(f"   • {member}")
    
print("\\n🔒 Loading environment variables for secure database connection...")
print("📝 Make sure you have created a .env file with your Supabase credentials")

## Part 1: Data Foundation (20 points)

### 1.1 Database Connection and Schema Exploration

**Tasks**:
1. Establish secure PostgreSQL connection to Supabase
2. Document all available tables and their relationships
3. Analyze data completeness and quality issues
4. Create a comprehensive data dictionary

**Evaluation Criteria**:
- ✅ Successful database connection with proper error handling
- ✅ Complete schema documentation
- ✅ Data quality assessment with quantified issues
- ✅ Professional data dictionary with business context

In [ ]:
# 1.1 Database Connection Setup

# Load environment variables from .env file
load_dotenv()

# Supabase PostgreSQL Configuration
DATABASE_CONFIG = {
    'host': os.getenv('POSTGRES_HOST'),
    'port': int(os.getenv('POSTGRES_PORT', 6543)),
    'database': os.getenv('POSTGRES_DATABASE', 'postgres'),
    'user': os.getenv('POSTGRES_USER'),
    'password': os.getenv('POSTGRES_PASSWORD')
}

# Verify that environment variables were loaded
if not all([DATABASE_CONFIG['host'], DATABASE_CONFIG['user'], DATABASE_CONFIG['password']]):
    print("❌ Missing required database credentials. Please check your .env file.")
    print("📝 Required variables: POSTGRES_HOST, POSTGRES_USER, POSTGRES_PASSWORD")
    print("💡 See the .env.example file for the correct format")
    raise ValueError("Database credentials not found in environment variables")

def create_database_connection():
    """
    Create and test PostgreSQL database connection.
    
    Returns:
    --------
    sqlalchemy.engine.Engine: Database engine
    """
    try:
        # Create PostgreSQL connection string
        connection_string = f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"
        
        # Create engine with connection pooling
        engine = create_engine(
            connection_string,
            pool_size=5,
            max_overflow=10,
            pool_timeout=30,
            pool_recycle=3600,
            connect_args={
                "connect_timeout": 30,
                "application_name": "Python_Data_Analysis_Course"
            }
        )
        
        # Test connection
        with engine.connect() as conn:
            result = conn.execute(text("SELECT version()"))
            version = result.scalar()
            print("✅ PostgreSQL connection established successfully")
            print(f"🐘 Database version: {version[:50]}...")
        
        return engine
        
    except Exception as e:
        print(f"❌ PostgreSQL connection failed: {e}")
        print("🔧 Troubleshooting tips:")
        print("  • Check your internet connection")
        print("  • Verify database credentials in .env file")
        print("  • Ensure Supabase database is running")
        print("  • Check if you need to whitelist your IP address")
        raise

# Test connection
print("🔗 Testing database connection...")
db_engine = create_database_connection()

In [ ]:
# 1.2 Schema Exploration and Documentation

def explore_database_schema(engine):
    """
    Comprehensive database schema exploration.
    
    Explores the actual Olist e-commerce database structure:
    - olist_sales_data_set: Core business tables
    - olist_marketing_data_set: Marketing and leads data
    
    Returns:
    --------
    dict: Comprehensive schema information
    """
    inspector = inspect(engine)
    schema_info = {}
    
    # Define the schemas we're working with
    target_schemas = ['olist_sales_data_set', 'olist_marketing_data_set']
    
    print("📋 Discovering Olist database schema...")
    
    for schema in target_schemas:
        try:
            tables = inspector.get_table_names(schema=schema)
            print(f"\\n📊 Schema '{schema}' contains {len(tables)} tables:")
            
            for table in tables:
                schema_qualified_name = f"{schema}.{table}"
                try:
                    with engine.connect() as conn:
                        # Get row count
                        result = conn.execute(text(f'SELECT COUNT(*) FROM "{schema}"."{table}"'))
                        row_count = result.scalar()
                        
                        # Get column information
                        columns = inspector.get_columns(table, schema=schema)
                        
                        schema_info[schema_qualified_name] = {
                            'schema': schema,
                            'table': table,
                            'rows': row_count,
                            'columns': [col['name'] for col in columns],
                            'column_types': {col['name']: str(col['type']) for col in columns}
                        }
                        
                        print(f"  📋 {table}: {row_count:,} rows, {len(columns)} columns")
                        
                except Exception as e:
                    print(f"  ⚠️ Could not access {schema}.{table}: {e}")
                    continue
                    
        except Exception as e:
            print(f"⚠️ Could not access schema '{schema}': {e}")
            continue
    
    # Display schema summary
    total_rows = sum(info['rows'] for info in schema_info.values())
    print(f"\\n🗃️ Total tables discovered: {len(schema_info)}")
    print(f"📏 Total rows across all tables: {total_rows:,}")
    
    # Show table relationships for business understanding
    print(f"\\n🔗 Key Business Relationships:")
    print("  • customers → orders → order_items → products")
    print("  • orders → payments & reviews")
    print("  • marketing_leads → closed_deals → sellers")
    print("  • geographic coordinates for Brazilian analysis")
    
    return schema_info

# Execute schema exploration
print("📊 Exploring database schema...")
schema_info = explore_database_schema(db_engine)

# Show sample of main business table
print("\\n📋 Sample from main orders table:")
sample_query = '''
SELECT 
    order_id,
    customer_id,
    order_status,
    order_purchase_timestamp,
    order_delivered_customer_date
FROM "olist_sales_data_set"."olist_orders_dataset" 
LIMIT 3
'''

try:
    sample_data = pd.read_sql(text(sample_query), db_engine)
    print("✅ Sample orders data:")
    display(sample_data)
except Exception as e:
    print(f"❌ Could not fetch sample data: {e}")

In [ ]:
# 1.3 Data Quality Assessment

def assess_data_quality(engine):
    """
    Comprehensive data quality assessment using actual Olist data.
    
    Analyzes:
    1. Missing values by table and column
    2. Duplicate records
    3. Data consistency across relationships
    4. Date range validation
    5. Business rule compliance
    
    Returns:
    --------
    dict: Data quality report with scores and recommendations
    """
    
    quality_report = {
        'completeness': {},
        'consistency': {},
        'validity': {},
        'overall_score': 0
    }
    
    print("🔍 Assessing data quality across Olist dataset...")
    
    # Check completeness for key business tables
    completeness_checks = {
        'customers': '''
            SELECT 
                COUNT(*) as total_records,
                COUNT(customer_id) as customer_id_filled,
                COUNT(customer_state) as state_filled,
                COUNT(customer_city) as city_filled
            FROM "olist_sales_data_set"."olist_customers_dataset"
        ''',
        'orders': '''
            SELECT 
                COUNT(*) as total_records,
                COUNT(order_id) as order_id_filled,
                COUNT(customer_id) as customer_id_filled,
                COUNT(order_purchase_timestamp) as purchase_date_filled,
                COUNT(order_status) as status_filled
            FROM "olist_sales_data_set"."olist_orders_dataset"
        ''',
        'reviews': '''
            SELECT 
                COUNT(*) as total_records,
                COUNT(review_id) as review_id_filled,
                COUNT(order_id) as order_id_filled,
                COUNT(review_score) as score_filled,
                COUNT(review_creation_date) as date_filled
            FROM "olist_sales_data_set"."olist_order_reviews_dataset"
        '''
    }
    
    print("\\n📊 Data Completeness Analysis:")
    for table_name, query in completeness_checks.items():
        try:
            result = pd.read_sql(text(query), engine)
            total = result.iloc[0]['total_records']
            
            print(f"\\n  📋 {table_name.title()} Table:")
            print(f"    Total records: {total:,}")
            
            completeness_scores = []
            for col in result.columns:
                if col != 'total_records':
                    filled = result.iloc[0][col]
                    completeness = (filled / total) * 100 if total > 0 else 0
                    completeness_scores.append(completeness)
                    print(f"    {col}: {completeness:.1f}% complete")
            
            quality_report['completeness'][table_name] = sum(completeness_scores) / len(completeness_scores)
            
        except Exception as e:
            print(f"    ❌ Could not assess {table_name}: {e}")
    
    # Check data consistency
    print("\\n🔗 Data Consistency Checks:")
    
    consistency_checks = {
        'customer_order_relationship': '''
            SELECT 
                COUNT(DISTINCT o.customer_id) as orders_customers,
                COUNT(DISTINCT c.customer_id) as total_customers,
                ROUND(COUNT(DISTINCT o.customer_id) * 100.0 / COUNT(DISTINCT c.customer_id), 1) as customer_engagement_rate
            FROM "olist_sales_data_set"."olist_customers_dataset" c
            LEFT JOIN "olist_sales_data_set"."olist_orders_dataset" o ON c.customer_id = o.customer_id
        ''',
        'order_items_integrity': '''
            SELECT 
                COUNT(DISTINCT oi.order_id) as orders_with_items,
                COUNT(DISTINCT o.order_id) as total_orders,
                ROUND(COUNT(DISTINCT oi.order_id) * 100.0 / COUNT(DISTINCT o.order_id), 1) as order_completion_rate
            FROM "olist_sales_data_set"."olist_orders_dataset" o
            LEFT JOIN "olist_sales_data_set"."olist_order_items_dataset" oi ON o.order_id = oi.order_id
        '''
    }
    
    for check_name, query in consistency_checks.items():
        try:
            result = pd.read_sql(text(query), engine)
            print(f"  ✅ {check_name.replace('_', ' ').title()}:")
            for col in result.columns:
                value = result.iloc[0][col]
                print(f"    {col}: {value}")
        except Exception as e:
            print(f"  ❌ {check_name} failed: {e}")
    
    # Check business logic validity
    print("\\n📈 Business Logic Validation:")
    
    validity_checks = {
        'review_scores': '''
            SELECT 
                MIN(review_score) as min_score,
                MAX(review_score) as max_score,
                COUNT(CASE WHEN review_score BETWEEN 1 AND 5 THEN 1 END) as valid_scores,
                COUNT(*) as total_scores
            FROM "olist_sales_data_set"."olist_order_reviews_dataset"
            WHERE review_score IS NOT NULL
        ''',
        'delivery_logic': '''
            SELECT 
                COUNT(CASE WHEN order_delivered_customer_date >= order_purchase_timestamp THEN 1 END) as logical_deliveries,
                COUNT(CASE WHEN order_delivered_customer_date IS NOT NULL AND order_purchase_timestamp IS NOT NULL THEN 1 END) as total_with_dates
            FROM "olist_sales_data_set"."olist_orders_dataset"
        '''
    }
    
    validity_scores = []
    for check_name, query in validity_checks.items():
        try:
            result = pd.read_sql(text(query), engine)
            print(f"  ✅ {check_name.replace('_', ' ').title()}:")
            
            if 'valid_scores' in result.columns:
                valid = result.iloc[0]['valid_scores']
                total = result.iloc[0]['total_scores']
                score = (valid / total) * 100 if total > 0 else 0
                validity_scores.append(score)
                print(f"    Valid review scores: {score:.1f}% ({valid:,}/{total:,})")
            
            if 'logical_deliveries' in result.columns:
                logical = result.iloc[0]['logical_deliveries']
                total = result.iloc[0]['total_with_dates']
                score = (logical / total) * 100 if total > 0 else 0
                validity_scores.append(score)
                print(f"    Logical delivery dates: {score:.1f}% ({logical:,}/{total:,})")
                
        except Exception as e:
            print(f"  ❌ {check_name} failed: {e}")
    
    # Calculate overall quality score
    completeness_avg = sum(quality_report['completeness'].values()) / len(quality_report['completeness']) if quality_report['completeness'] else 0
    validity_avg = sum(validity_scores) / len(validity_scores) if validity_scores else 0
    overall_score = (completeness_avg + validity_avg) / 2
    
    quality_report['overall_score'] = overall_score
    
    print(f"\\n📊 Data Quality Summary:")
    print(f"  • Completeness Score: {completeness_avg:.1f}/100")
    print(f"  • Validity Score: {validity_avg:.1f}/100") 
    print(f"  • Overall Quality Score: {overall_score:.1f}/100")
    
    if overall_score >= 90:
        print("  🎉 Excellent data quality - ready for analysis!")
    elif overall_score >= 80:
        print("  ✅ Good data quality - minor issues to address")
    elif overall_score >= 70:
        print("  ⚠️ Acceptable data quality - some attention needed")
    else:
        print("  ❌ Poor data quality - significant cleanup required")
    
    return quality_report

# Execute data quality assessment
print("🔍 Assessing data quality...")
quality_report = assess_data_quality(db_engine)

### 1.4 Data Quality Report

**Document your findings here**:

#### Table Inventory
| Table Name | Row Count | Key Columns | Business Purpose |
|------------|-----------|-------------|------------------|
| | | | |

#### Data Quality Issues
| Issue Type | Affected Tables | Severity | Recommended Action |
|------------|----------------|----------|--------------------|
| | | | |

#### Data Quality Score: __/100

**Executive Summary**: Write 2-3 sentences summarizing the overall data quality and readiness for analysis.

## Part 2: Advanced Analytics (35 points)

### 2.1 Customer Segmentation Analysis (Advanced Pandas)

**Business Question**: "Which customer segments drive the most value, and how should we tailor our strategies to each segment?"

**Requirements**:
1. Implement RFM (Recency, Frequency, Monetary) analysis using advanced pandas
2. Create customer lifetime value predictions
3. Identify high-value customer characteristics
4. Develop customer churn risk scoring

**Advanced Pandas Techniques to Demonstrate**:
- Multi-level groupby operations
- Window functions and rolling calculations
- Advanced indexing and data transformation
- Statistical analysis and correlation studies

In [ ]:
# 2.1 Customer Segmentation with Advanced Pandas

def extract_customer_data(engine):
    """
    Extract comprehensive customer data using SQL.
    
    TODO: Create SQL query that joins:
    - Customer information
    - Order history
    - Purchase behavior
    - Geographic data
    - Review scores
    
    Returns:
    --------
    pd.DataFrame: Customer dataset ready for analysis
    """
    
    # TODO: Write comprehensive SQL query
    sql_query = """
    -- Your comprehensive customer analysis query here
    -- Think about what data you need for RFM analysis
    """
    
    # Execute query and return DataFrame
    # return pd.read_sql(sql_query, engine)
    pass

def perform_rfm_analysis(customer_df):
    """
    Advanced RFM analysis using pandas.
    
    TODO: Implement:
    1. Recency calculation (days since last purchase)
    2. Frequency calculation (number of orders)
    3. Monetary calculation (total customer value)
    4. RFM scoring (1-5 scale for each dimension)
    5. Customer segmentation based on RFM scores
    
    Advanced pandas techniques:
    - Use pd.qcut() for quintile-based scoring
    - Apply custom scoring functions
    - Create multi-level segments
    
    Returns:
    --------
    pd.DataFrame: Customer data with RFM scores and segments
    """
    # Your implementation here
    pass

def calculate_customer_lifetime_value(customer_df):
    """
    Calculate CLV using advanced pandas operations.
    
    TODO: Implement CLV calculation using:
    - Historical purchase patterns
    - Cohort analysis
    - Retention rate modeling
    - Predictive CLV scoring
    
    Returns:
    --------
    pd.DataFrame: Customer data with CLV predictions
    """
    # Your implementation here
    pass

# Execute customer segmentation analysis
print("👥 Performing customer segmentation analysis...")
# customer_data = extract_customer_data(db_engine)
# rfm_analysis = perform_rfm_analysis(customer_data)
# clv_analysis = calculate_customer_lifetime_value(customer_data)

In [ ]:
# 2.1.2 Advanced Customer Analytics

def analyze_customer_behavior_patterns(customer_df):
    """
    Advanced behavioral analysis using pandas.
    
    TODO: Implement:
    1. Purchase seasonality analysis
    2. Product preference mapping
    3. Geographic behavior patterns
    4. Customer journey analysis
    5. Churn prediction scoring
    
    Advanced pandas techniques:
    - Time series analysis with groupby
    - Rolling window calculations
    - Cross-tabulation and pivot operations
    - Statistical correlation analysis
    """
    # Your implementation here
    pass

# TODO: Create interactive customer segmentation dashboard
def create_customer_dashboard(customer_analysis):
    """
    Create interactive Plotly dashboard for customer insights.
    
    Include:
    - RFM segment distribution
    - CLV predictions by segment
    - Geographic customer value mapping
    - Customer journey visualization
    """
    # Your dashboard implementation here
    pass

### 2.2 Marketing Channel ROI Analysis (SQL-Python Integration)

**Business Question**: "Which marketing channels provide the best return on investment, and how should we optimize our marketing budget allocation?"

**Requirements**:
1. Combine marketing funnel data with customer value using SQL joins
2. Calculate channel-specific conversion rates and customer acquisition costs
3. Analyze customer lifetime value by acquisition channel
4. Build ROI optimization model

**SQL-Python Integration Techniques**:
- Complex SQL queries with CTEs and window functions
- Parameterized queries for dynamic analysis
- Combining SQL aggregation with pandas statistical analysis
- Performance optimization strategies

In [ ]:
# 2.2 Marketing Channel Analysis

def analyze_marketing_funnel_performance(engine):
    """
    Comprehensive marketing funnel analysis using advanced SQL.
    
    TODO: Create SQL analysis that includes:
    1. Lead-to-customer conversion rates by channel
    2. Customer acquisition cost calculation
    3. Channel performance over time
    4. Customer quality metrics by channel
    
    Use advanced SQL features:
    - CTEs for complex multi-step calculations
    - Window functions for running totals and rankings
    - CASE statements for business logic
    - Date functions for cohort analysis
    """
    
    # TODO: Write advanced SQL query
    marketing_analysis_sql = """
    WITH marketing_funnel AS (
        -- Your comprehensive marketing funnel query here
        -- Include lead generation, conversion, and customer value data
    ),
    channel_performance AS (
        -- Calculate channel-specific metrics
    ),
    roi_analysis AS (
        -- ROI calculations by channel
    )
    SELECT 
        -- Your final results here
    FROM roi_analysis
    ORDER BY roi DESC
    """
    
    # return pd.read_sql(marketing_analysis_sql, engine)
    pass

def calculate_customer_acquisition_metrics(engine):
    """
    Calculate detailed customer acquisition metrics.
    
    TODO: Implement SQL-Python hybrid approach:
    1. Use SQL for data extraction and initial aggregation
    2. Use pandas for advanced statistical analysis
    3. Calculate CAC, LTV, and ROI by channel
    4. Perform cohort analysis
    """
    # Your implementation here
    pass

# Execute marketing analysis
print("📈 Analyzing marketing channel performance...")
# marketing_performance = analyze_marketing_funnel_performance(db_engine)
# acquisition_metrics = calculate_customer_acquisition_metrics(db_engine)

### 2.3 Geographic Market Analysis

**Business Question**: "Which geographic markets offer the biggest growth opportunities, and what market entry strategies should we pursue?"

**Requirements**:
1. Analyze market penetration by state and city
2. Identify underperforming regions with high potential
3. Calculate market size and competition analysis
4. Develop geographic expansion scoring model

In [ ]:
# 2.3 Geographic Market Intelligence

def analyze_geographic_performance(engine):
    """
    Comprehensive geographic market analysis.
    
    TODO: Implement analysis including:
    1. Revenue and customer density by state/city
    2. Market penetration rates
    3. Average order values by region
    4. Growth rate analysis
    5. Delivery performance by geography
    """
    # Your implementation here
    pass

def identify_expansion_opportunities(geographic_data):
    """
    Identify high-potential expansion markets.
    
    TODO: Create scoring model based on:
    1. Market size vs current penetration
    2. Customer value potential
    3. Competitive landscape
    4. Operational feasibility
    """
    # Your implementation here
    pass

# TODO: Create geographic visualization dashboard
def create_geographic_dashboard(geo_analysis):
    """
    Create interactive geographic dashboard.
    
    Include:
    - Choropleth maps of market performance
    - Bubble charts for opportunity sizing
    - Time series of regional growth
    """
    # Your dashboard implementation here
    pass

### 2.4 Product Portfolio Optimization

**Business Question**: "Which product categories should we prioritize for investment, and what products show the most promise for growth?"

**Requirements**:
1. Analyze product category performance trends
2. Identify emerging product opportunities
3. Calculate product profitability and customer satisfaction
4. Build product recommendation engine logic

In [ ]:
# 2.4 Product Portfolio Analysis

def analyze_product_performance(engine):
    """
    Comprehensive product and category analysis.
    
    TODO: Analyze:
    1. Category revenue and growth trends
    2. Product satisfaction scores
    3. Cross-selling opportunities
    4. Seasonal demand patterns
    5. Inventory turnover analysis
    """
    # Your implementation here
    pass

def build_product_recommendation_engine(product_data, customer_data):
    """
    Build product recommendation logic.
    
    TODO: Implement:
    1. Customer purchase pattern analysis
    2. Product affinity scoring
    3. Collaborative filtering basics
    4. Business rule integration
    """
    # Your implementation here
    pass

# Execute product analysis
print("🛍️ Analyzing product portfolio performance...")
# product_analysis = analyze_product_performance(db_engine)
# recommendation_engine = build_product_recommendation_engine(product_analysis, customer_data)

## Part 3: Interactive Business Intelligence Dashboard (25 points)

### 3.1 Executive Dashboard Creation

**Requirements**:
1. Create comprehensive interactive dashboard with 5+ visualizations
2. Implement cross-chart filtering and interactions
3. Include key performance indicators (KPIs)
4. Design for executive-level presentation

**Dashboard Components**:
- Executive KPI summary
- Customer segmentation insights
- Marketing channel performance
- Geographic market opportunities
- Product portfolio analysis
- Trend analysis and forecasting

In [ ]:
# 3.1 Executive Dashboard

def create_executive_dashboard(customer_analysis, marketing_analysis, geographic_analysis, product_analysis):
    """
    Create comprehensive executive dashboard.
    
    TODO: Build interactive dashboard with:
    1. KPI summary cards
    2. Multi-chart layout with subplots
    3. Interactive filters and controls
    4. Professional styling and branding
    5. Responsive design
    
    Dashboard sections:
    - Executive Summary (KPIs)
    - Customer Intelligence
    - Marketing Performance
    - Geographic Insights
    - Product Analysis
    """
    
    # TODO: Create multi-section dashboard
    
    # Section 1: Executive KPIs
    def create_kpi_section():
        # Create KPI cards and summary metrics
        pass
    
    # Section 2: Customer Intelligence
    def create_customer_section():
        # Customer segmentation and behavior visualizations
        pass
    
    # Section 3: Marketing Performance
    def create_marketing_section():
        # Channel performance and ROI analysis
        pass
    
    # Section 4: Geographic Intelligence
    def create_geographic_section():
        # Market opportunity and performance maps
        pass
    
    # Section 5: Product Portfolio
    def create_product_section():
        # Product performance and recommendations
        pass
    
    # Combine all sections
    # Your dashboard implementation here
    pass

def create_interactive_filters(dashboard):
    """
    Add interactive filters and controls.
    
    TODO: Implement:
    1. Date range selectors
    2. Geographic filters
    3. Customer segment filters
    4. Product category filters
    5. Cross-chart interactions
    """
    # Your filter implementation here
    pass

# Create and display executive dashboard
print("📊 Creating executive business intelligence dashboard...")
# executive_dashboard = create_executive_dashboard(
#     customer_analysis, marketing_analysis, geographic_analysis, product_analysis
# )

### 3.2 Advanced Visualization Techniques

**Demonstrate advanced Plotly features**:
- Animation for temporal analysis
- 3D visualizations for multi-dimensional data
- Custom hover templates and annotations
- Advanced color schemes and styling
- Responsive layouts

In [ ]:
# 3.2 Advanced Visualizations

def create_advanced_visualizations():
    """
    Showcase advanced Plotly visualization techniques.
    
    TODO: Create:
    1. Animated time series showing business evolution
    2. 3D customer segmentation visualization
    3. Interactive geographic choropleth with drill-down
    4. Advanced financial charts (candlestick, waterfall)
    5. Network graph for product relationships
    """
    
    # Animation: Business metrics over time
    def create_animated_business_evolution():
        # Your animated visualization here
        pass
    
    # 3D: Customer segmentation space
    def create_3d_customer_analysis():
        # Your 3D visualization here
        pass
    
    # Geographic: Interactive market analysis
    def create_interactive_map():
        # Your geographic visualization here
        pass
    
    # Financial: Revenue waterfall analysis
    def create_financial_analysis():
        # Your financial visualization here
        pass
    
    # Your implementation here
    pass

# Create advanced visualizations
print("🎨 Creating advanced visualizations...")
# advanced_visuals = create_advanced_visualizations()

## Part 4: Strategic Recommendations (20 points)

### 4.1 Business Intelligence Synthesis

**Requirements**:
1. Synthesize insights from all analyses
2. Quantify business impact and ROI
3. Prioritize recommendations by impact and feasibility
4. Create implementation roadmap

**Deliverables**:
- Executive summary of key findings
- Prioritized recommendation list
- ROI projections and business case
- Implementation timeline

In [ ]:
# 4.1 Strategic Analysis and Recommendations

def synthesize_business_insights():
    """
    Synthesize all analyses into actionable business insights.
    
    TODO: Create comprehensive analysis including:
    1. Key performance drivers identification
    2. Growth opportunity quantification
    3. Risk assessment and mitigation strategies
    4. Competitive advantage analysis
    5. Resource requirement estimation
    """
    
    business_insights = {
        'customer_strategy': {
            'key_segments': [],
            'growth_opportunities': [],
            'retention_strategies': [],
            'revenue_impact': 0
        },
        'marketing_optimization': {
            'top_channels': [],
            'budget_reallocation': {},
            'new_channel_opportunities': [],
            'roi_improvement': 0
        },
        'geographic_expansion': {
            'priority_markets': [],
            'entry_strategies': {},
            'investment_requirements': {},
            'revenue_potential': 0
        },
        'product_portfolio': {
            'priority_categories': [],
            'discontinuation_candidates': [],
            'new_product_opportunities': [],
            'margin_improvement': 0
        }
    }
    
    # TODO: Populate insights from your analyses
    
    return business_insights

def calculate_business_impact(insights):
    """
    Quantify the business impact of recommendations.
    
    TODO: Calculate:
    1. Revenue impact projections
    2. Cost savings opportunities
    3. ROI estimates
    4. Payback periods
    5. Risk-adjusted returns
    """
    # Your impact calculation here
    pass

# Generate strategic insights
print("🎯 Synthesizing strategic business insights...")
business_insights = synthesize_business_insights()
# business_impact = calculate_business_impact(business_insights)

### 4.2 Executive Summary and Recommendations

**Complete this section with your team's findings and recommendations**:

## 🎯 Executive Summary

### Key Findings

**1. Customer Strategy**
- *Finding*: [Your key customer insights here]
- *Impact*: [Quantified business impact]
- *Recommendation*: [Specific actionable recommendation]

**2. Marketing Optimization**
- *Finding*: [Your marketing channel insights here]
- *Impact*: [ROI improvement potential]
- *Recommendation*: [Budget reallocation strategy]

**3. Geographic Expansion**
- *Finding*: [Your geographic opportunities here]
- *Impact*: [Market size and revenue potential]
- *Recommendation*: [Expansion strategy and timeline]

**4. Product Portfolio**
- *Finding*: [Your product insights here]
- *Impact*: [Margin and growth improvement]
- *Recommendation*: [Portfolio optimization strategy]

### Strategic Recommendations

#### Priority 1: High Impact, Low Effort
1. **[Recommendation Title]**
   - *Description*: [What to do]
   - *Business Impact*: [Quantified benefit]
   - *Implementation*: [How to execute]
   - *Timeline*: [When to complete]
   - *Investment Required*: [Resources needed]

#### Priority 2: High Impact, Medium Effort
2. **[Recommendation Title]**
   - *Description*: [What to do]
   - *Business Impact*: [Quantified benefit]
   - *Implementation*: [How to execute]
   - *Timeline*: [When to complete]
   - *Investment Required*: [Resources needed]

#### Priority 3: High Impact, High Effort
3. **[Recommendation Title]**
   - *Description*: [What to do]
   - *Business Impact*: [Quantified benefit]
   - *Implementation*: [How to execute]
   - *Timeline*: [When to complete]
   - *Investment Required*: [Resources needed]

### Financial Projections

| Recommendation | Year 1 Impact | Year 2 Impact | Total ROI | Implementation Cost |
|----------------|---------------|---------------|-----------|--------------------|
| [Rec 1] | $X.X M | $X.X M | X% | $X.X M |
| [Rec 2] | $X.X M | $X.X M | X% | $X.X M |
| [Rec 3] | $X.X M | $X.X M | X% | $X.X M |
| **Total** | **$X.X M** | **$X.X M** | **X%** | **$X.X M** |

### Implementation Roadmap

#### Q1 2024
- [Action items for Q1]

#### Q2 2024
- [Action items for Q2]

#### Q3 2024
- [Action items for Q3]

#### Q4 2024
- [Action items for Q4]

### Success Metrics

**Customer Strategy Success Metrics:**
- [Specific KPIs to track]

**Marketing Success Metrics:**
- [Specific KPIs to track]

**Geographic Success Metrics:**
- [Specific KPIs to track]

**Product Success Metrics:**
- [Specific KPIs to track]

### Risk Mitigation

**Key Risks and Mitigation Strategies:**
1. *Risk*: [Identified risk] | *Mitigation*: [How to address]
2. *Risk*: [Identified risk] | *Mitigation*: [How to address]
3. *Risk*: [Identified risk] | *Mitigation*: [How to address]

## 📊 Assignment Evaluation Rubric

### Part 1: Data Foundation (20 points)
- **Excellent (18-20)**: Robust database connection, comprehensive schema documentation, thorough data quality assessment
- **Good (15-17)**: Solid connection setup, good schema exploration, adequate quality analysis
- **Satisfactory (12-14)**: Basic connection, limited exploration, minimal quality assessment
- **Needs Improvement (0-11)**: Connection issues, incomplete exploration, poor quality analysis

### Part 2: Advanced Analytics (35 points)
- **Excellent (32-35)**: Sophisticated pandas operations, advanced SQL integration, insightful business analysis
- **Good (28-31)**: Good analytical techniques, solid SQL-pandas integration, meaningful insights
- **Satisfactory (21-27)**: Basic analysis, simple integration, limited insights
- **Needs Improvement (0-20)**: Poor analysis, minimal integration, unclear insights

### Part 3: Interactive Visualizations (25 points)
- **Excellent (23-25)**: Professional dashboard, advanced interactivity, excellent design
- **Good (20-22)**: Solid dashboard, good interactivity, clear visualizations
- **Satisfactory (15-19)**: Basic dashboard, limited interactivity, acceptable design
- **Needs Improvement (0-14)**: Poor visualizations, minimal interactivity, unclear design

### Part 4: Strategic Recommendations (20 points)
- **Excellent (18-20)**: Actionable insights, quantified impact, clear implementation plan
- **Good (15-17)**: Good insights, estimated impact, reasonable implementation
- **Satisfactory (12-14)**: Basic insights, limited impact analysis, vague implementation
- **Needs Improvement (0-11)**: Poor insights, no impact analysis, unclear implementation

### Additional Evaluation Criteria:
- **Code Quality**: Clean, documented, efficient code
- **Business Acumen**: Understanding of business context and implications
- **Technical Innovation**: Creative use of advanced techniques
- **Presentation**: Professional presentation and communication
- **Teamwork**: Evidence of effective collaboration

### Total Score: ___/100

## 📝 Submission Instructions

### Deliverables Checklist:
- [ ] **Completed Jupyter Notebook** (this file) with all code and analysis
- [ ] **Executive Summary Report** (2-3 page PDF) with key findings and recommendations
- [ ] **Group Presentation** (10 minutes) showcasing insights and dashboard
- [ ] **Individual Reflection** (1 page per team member) on learning and contribution

### Technical Requirements:
- [ ] All code cells execute without errors
- [ ] Database connections are properly managed and closed
- [ ] Visualizations render correctly in the notebook
- [ ] Code is well-documented with comments and docstrings
- [ ] Data sources are properly cited

### Presentation Guidelines:
- **Duration**: 10 minutes + 5 minutes Q&A
- **Format**: Professional business presentation
- **Content**: Focus on insights and recommendations, not technical details
- **Audience**: Assume C-level executives as your audience
- **Demo**: Include live dashboard demonstration

### Submission Details:
- **Due Date**: [Insert due date]
- **Submission Method**: [Insert submission instructions]
- **File Naming**: `Week6_Major_Assignment_[TeamName].ipynb`
- **Late Policy**: [Insert late policy]

### Support Resources:
- **Office Hours**: [Insert schedule]
- **Discussion Forum**: [Insert link]
- **Technical Support**: [Insert contact]

---

## 🎉 Good Luck!

This assignment represents the culmination of your SQL-Python integration learning. Focus on:
- **Business Impact**: Always connect technical findings to business value
- **Collaboration**: Leverage each team member's strengths
- **Innovation**: Don't be afraid to try advanced techniques
- **Communication**: Tell compelling stories with your data

Remember: The goal is not just to analyze data, but to generate insights that drive real business decisions. Think like consultants delivering recommendations to Olist's executive team.

**Happy analyzing! 🚀📊**